In [7]:
import os
import cv2
import numpy as np
from skimage.util import random_noise

## Costanti

In [8]:
PATH_DATASET_LAVORATO = './Dati Sensori/Dataset Sensori Lavorato'
PATH_DATASET_NOISE = './Dati Sensori/Dataset Sensori Noise'
PATH_DATASET_ART_RIF = './Dati Sensori/Dataset Sensori Artefatti Riflessione'
PATH_DATASET_SPECKLE = './Dati Sensori/Dataset Sensori Speckle'

## Configurazione

In [33]:
def genera_immagini_distorte(dir_img_orig, dir_img_dist, callback_distorsione, **kwargs):
    for root, dirs, files in os.walk(dir_img_orig):
        for file in files:
            if not file.endswith(".jpg"): continue
            path_img_orig = os.path.join(root, file)
            path_img_dist = path_img_orig.replace(dir_img_orig, dir_img_dist)
            if ("speckle_mean" in kwargs) and ("speckle_std" in kwargs):
                speckle_mean = kwargs['speckle_mean']
                speckle_std = kwargs['speckle_std']
                print(f"** Applico speckle: M{speckle_mean}-S{speckle_std}")
                callback_distorsione(path_img_orig, path_img_dist, speckle_mean, speckle_std)
            elif "peso_img_rifl" in kwargs:
                peso_img_rifl = kwargs['peso_img_rifl']
                print(f"** Applico riflessione: P{peso_img_rifl}")
                callback_distorsione(path_img_orig, path_img_dist, peso_img_rifl)
            

## Rumore gaussiano

## Artefatti di riflessione
Vengono modificati i valori dei pixel per riflettere l'immagine rispetto all'asse di simmetria.
L'immagine riflessa viene sovrapposta all'immagine originale per ottenere un effetto di fusione tra le due immagini.  
vd. https://onlinelibrary.wiley.com/doi/abs/10.7863/jum.1986.5.4.227

In [25]:
def applica_artefatti_riflessione(path_img_orig, path_img_dist, peso_img_rifl):
        print(f"Applico artefatti di riflessione a {path_img_orig} ...")
        img = cv2.imread(path_img_orig)
        img_riflessa = img.copy()
        h, w, _ = img.shape
        centro_x = int(w/2)
        for y in range(h):
            for x in range(centro_x, w):
                x_rifl = w - x - 1  # Posizione x del pixel riflesso
                img_riflessa[y, x] = img[y, x_rifl]
        blended_img = cv2.addWeighted(img, 1-peso_img_rifl, img_riflessa, peso_img_rifl, 0)
        os.makedirs(os.path.dirname(path_img_dist), exist_ok=True)
        cv2.imwrite(path_img_dist, blended_img)

In [29]:
%%time
nPesi = 7
pesi_img_riflessa = [round(0.1 * i, 2) for i in range(nPesi)]

for i, peso_rifl in enumerate(pesi_img_riflessa):
    dir_img_dist = f"{PATH_DATASET_ART_RIF}-{i}"
    genera_immagini_distorte(PATH_DATASET_LAVORATO, f"{PATH_DATASET_ART_RIF}-{i}", applica_artefatti_riflessione, peso_img_rifl=peso_rifl)

** Applico riflessione: P0.0
Applico artefatti di riflessione a ./Dati Sensori/Dataset Sensori Lavorato/9_M/9_M_1/9_M_1_img_3.jpg ...
** Applico riflessione: P0.0
Applico artefatti di riflessione a ./Dati Sensori/Dataset Sensori Lavorato/9_M/9_M_1/9_M_1_img_2.jpg ...
** Applico riflessione: P0.0
Applico artefatti di riflessione a ./Dati Sensori/Dataset Sensori Lavorato/9_M/9_M_1/9_M_1_img_1.jpg ...
** Applico riflessione: P0.0
Applico artefatti di riflessione a ./Dati Sensori/Dataset Sensori Lavorato/9_M/9_M_1/9_M_1_img_5.jpg ...
** Applico riflessione: P0.0
Applico artefatti di riflessione a ./Dati Sensori/Dataset Sensori Lavorato/9_M/9_M_1/9_M_1_img_4.jpg ...
** Applico riflessione: P0.0
Applico artefatti di riflessione a ./Dati Sensori/Dataset Sensori Lavorato/9_M/9_M_1/9_M_1_img_6.jpg ...
** Applico riflessione: P0.0
Applico artefatti di riflessione a ./Dati Sensori/Dataset Sensori Lavorato/9_M/9_M_1/9_M_1_img_7.jpg ...
** Applico riflessione: P0.0
Applico artefatti di riflessione 

## Speckle

In [30]:
def applica_speckle(path_img_orig, path_img_dist, speckle_mean, speckle_std):
    print(f"Applico lo speckle a {path_img_orig} ...")
    img = cv2.imread(path_img_orig, 0)
    speckle_noise = np.random.normal(speckle_mean, speckle_std, img.shape)
    img_rumore = img + img * speckle_noise
    img_rumore = cv2.normalize(img_rumore, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    os.makedirs(os.path.dirname(path_img_dist), exist_ok=True)
    cv2.imwrite(path_img_dist, img_rumore)

In [34]:
%%time
nStd = 5
nMean = 3

speckle_mean_list = [round(0.1 * i, 2) for i in range(nMean)]  
speckle_std_list = [round(0.1 * i, 2) for i in range(nStd)]

for i, speckle_mean in enumerate(speckle_mean_list):
    for j, speckle_std in enumerate(speckle_std_list):
        dir_img_dist = f"{PATH_DATASET_SPECKLE}-m{i}-s{j}"
        genera_immagini_distorte(PATH_DATASET_LAVORATO, dir_img_dist, applica_speckle, speckle_mean=speckle_mean, speckle_std=speckle_std)

** Applico speckle: M0.0-S0.0
Applico lo speckle a ./Dati Sensori/Dataset Sensori Lavorato/9_M/9_M_1/9_M_1_img_3.jpg ...
** Applico speckle: M0.0-S0.0
Applico lo speckle a ./Dati Sensori/Dataset Sensori Lavorato/9_M/9_M_1/9_M_1_img_2.jpg ...
** Applico speckle: M0.0-S0.0
Applico lo speckle a ./Dati Sensori/Dataset Sensori Lavorato/9_M/9_M_1/9_M_1_img_1.jpg ...
** Applico speckle: M0.0-S0.0
Applico lo speckle a ./Dati Sensori/Dataset Sensori Lavorato/9_M/9_M_1/9_M_1_img_5.jpg ...
** Applico speckle: M0.0-S0.0
Applico lo speckle a ./Dati Sensori/Dataset Sensori Lavorato/9_M/9_M_1/9_M_1_img_4.jpg ...
** Applico speckle: M0.0-S0.0
Applico lo speckle a ./Dati Sensori/Dataset Sensori Lavorato/9_M/9_M_1/9_M_1_img_6.jpg ...
** Applico speckle: M0.0-S0.0
Applico lo speckle a ./Dati Sensori/Dataset Sensori Lavorato/9_M/9_M_1/9_M_1_img_7.jpg ...
** Applico speckle: M0.0-S0.0
Applico lo speckle a ./Dati Sensori/Dataset Sensori Lavorato/9_M/9_M_1/9_M_1_img_10.jpg ...
** Applico speckle: M0.0-S0.0
A